In [55]:
# Required Libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# === CONFIGURATION ===
kpi_file = "innov_kpi_summary.csv"
ccwm_file = "ccwm.csv"
kpi_df = pd.read_csv(kpi_file)
ccwm_df = pd.read_csv(ccwm_file)

# === KPIs to Plot (with exact header names) ===
selected_kpis = [
    'All Win Rate',
    'All Avg For',
    'Weighted Avg For',
    'Weighted Normalized Win Margin',
    'Regional+ Win Rate',
    'ccwm',
    'opr'
]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_alliance_vs_alliance(game_name, red_teams, blue_teams, target_team="na"):
    local_kpi_df = kpi_df.copy()
    local_ccwm_df = ccwm_df.copy()

    # Align team names
    local_kpi_df['Team'] = local_kpi_df['Team'].astype(str).str.strip().str.upper()
    local_ccwm_df['team_number'] = local_ccwm_df['team_number'].astype(str).str.strip().str.upper()
    local_ccwm_df.rename(columns={'team_number': 'Team'}, inplace=True)

    # Merge KPI and CCWM/OPR
    full_df = pd.merge(local_kpi_df, local_ccwm_df[['Team', 'ccwm', 'opr']], on='Team', how='left')

    red_teams = [t.strip().upper() for t in red_teams]
    blue_teams = [t.strip().upper() for t in blue_teams]
    team_subset = red_teams + blue_teams
    filtered_df = full_df[full_df['Team'].isin(team_subset)].copy()

    if filtered_df.empty:
        return None

    # Normalization Factors
    norm_factors = {}
    for kpi in ['All Win Rate', 'All Avg For', 'Weighted Avg For', 'Regional+ Win Rate', 'opr']:
        top_two = full_df[kpi].dropna().unique()
        top_two.sort()
        norm_factors[kpi] = top_two[-1] + top_two[-2] if len(top_two) >= 2 else top_two[0] * 2 if len(top_two) == 1 else 1
    for kpi in ['Weighted Normalized Win Margin', 'ccwm']:
        values = full_df[kpi].dropna().values
        pair_sums = [x + y for i, x in enumerate(values) for j, y in enumerate(values) if i != j]
        norm_factors[kpi] = (min(pair_sums), max(pair_sums)) if pair_sums else (0, 1)

    # Normalize
    normalized_df = filtered_df.copy()
    for kpi in selected_kpis:
        if kpi in ['Weighted Normalized Win Margin', 'ccwm']:
            min_val, max_val = norm_factors[kpi]
            normalized_df[kpi] = (filtered_df[kpi] + abs(min_val)) / (max_val + abs(min_val) + 1e-8)
        else:
            normalized_df[kpi] = filtered_df[kpi] / (norm_factors[kpi] + 1e-8)

    def compute_alliance_values(df, teams):
        alliance_df = df[df['Team'].isin(teams)]
        return [alliance_df[kpi].nlargest(min(2, len(alliance_df))).sum() for kpi in selected_kpis]

    if not any(t in filtered_df['Team'].values for t in red_teams) or not any(t in filtered_df['Team'].values for t in blue_teams):
        return None

    red_values = compute_alliance_values(normalized_df, red_teams)
    blue_values = compute_alliance_values(normalized_df, blue_teams)

    # === Radar Chart ===
    red_values.append(red_values[0])
    blue_values.append(blue_values[0])
    labels = selected_kpis
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles.append(angles[0])

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    ax.plot(angles, red_values, color='red', linewidth=2, label=f'Red: {red_teams[0]}, {red_teams[1]}')
    ax.fill(angles, red_values, color='red', alpha=0.25)

    ax.plot(angles, blue_values, color='blue', linewidth=2, label=f'Blue: {blue_teams[0]}, {blue_teams[1]}')
    ax.fill(angles, blue_values, color='blue', alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=12)
    ax.set_title(f"{game_name} KPI Radar", size=16, pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))

    plt.tight_layout()
    plt.savefig(f"{game_name}.png")
    plt.close()

    # === Radar Area Score ===
    red_array = np.array(red_values[:-1])
    blue_array = np.array(blue_values[:-1])
    angle_step = 2 * np.pi / len(red_array)

    area_red = 0.5 * np.sum(red_array**2 * angle_step)
    area_blue = 0.5 * np.sum(blue_array**2 * angle_step)
    signed_diff_area = area_red - area_blue

    if target_team in red_teams:
        return signed_diff_area / (area_red + 1e-8)
    elif target_team in blue_teams:
        return (-1) * signed_diff_area / (area_blue + 1e-8)
    else:
        return signed_diff_area


In [58]:
target_team = "86254B"

with open("inno_matches.csv", mode='r', encoding='utf-8') as csv_file:
    matches = csv.DictReader(csv_file)

    for match in matches:
        red_teams = [
            match.get('Red Team 1', ''),
            match.get('Red Team 2', '')
        ]
        blue_teams = [
            match.get('Blue Team 1', ''),
            match.get('Blue Team 2', '')
        ]
        if "30214A" in red_teams or "30214A" in blue_teams:
            continue
        if target_team not in red_teams and target_team not in blue_teams:
            continue
        
        alliance_teammate = "na"
        opposition = []
        if target_team in red_teams:
            for team in red_teams:
                if team != target_team:
                    alliance_teammate = team
                    opposition = blue_teams
        else:
            for team in blue_teams:
                if team != target_team:
                    alliance_teammate = team
                    opposition = red_teams

        advantage = plot_alliance_vs_alliance(match.get('Match Name'), red_teams, blue_teams, target_team)
        print(match.get('Match Name'), advantage)

Qualifier #6 -0.9751567366274829
Qualifier #41 0.4448810357262004
Qualifier #57 0.16331420190623885
Qualifier #78 0.13237462432079597
Qualifier #98 0.6549122252010591
Qualifier #123 -0.5881484446181414
Qualifier #165 0.36506947244682325
Qualifier #176 -0.5854612561607773
Qualifier #203 0.1087961666378961
